# Dataset 1: articles_dataset.csv

#### a. Import required libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#import library
import os
import random
import re
import string

import nltk
import numpy as np
import pandas as pd

from gensim.models import Word2Vec

from nltk import word_tokenize
from nltk.corpus import stopwords
from collections import Counter

from ds_utils.clustering import vectorize, mbkmeans_clusters
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_samples, silhouette_score

nltk.download("stopwords")

SEED = 42
random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
np.random.seed(SEED)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#### b. Data Preprocessing

In [ ]:
import pandas as pd
df = pd.read_csv("./drive/MyDrive/STBI/Proyek/Dataset/articles_data.csv")
df.head()

,Unnamed: 0,source_id,source_name,author,title,description,url,url_to_image,published_at,content,top_article,engagement_reaction_count,engagement_comment_count,engagement_share_count,engagement_comment_plugin_count
0,0,reuters,Reuters,Reuters Editorial,NTSB says Autopilot engaged in 2018 California...,The National Transportation Safety Board said ...,https://www.reuters.com/article/us-tesla-crash...,https://s4.reutersmedia.net/resources/r/?m=02&...,2019-09-03T16:22:20Z,WASHINGTON (Reuters) - The National Transporta...,0.0,0.0,0.0,2528.0,0.0
1,1,the-irish-times,The Irish Times,Eoin Burke-Kennedy,Unemployment falls to post-crash low of 5.2%,Latest monthly figures reflect continued growt...,https://www.irishtimes.com/business/economy/un...,https://www.irishtimes.com/image-creator/?id=1...,2019-09-03T10:32:28Z,The States jobless rate fell to 5.2 per cent l...,0.0,6.0,10.0,2.0,0.0
2,2,the-irish-times,The Irish Times,Deirdre McQuillan,"Louise Kennedy AW2019: Long coats, sparkling t...",Autumn-winter collection features designer’s g...,https://www.irishtimes.com/\t\t\t\t\t\t\t/life...,https://www.irishtimes.com/image-creator/?id=1...,2019-09-03T14:40:00Z,Louise Kennedy is showing off her autumn-winte...,1.0,NaN,NaN,NaN,NaN
3,3,al-jazeera-english,Al Jazeera English,Al Jazeera,North Korean footballer Han joins Italian gian...,Han is the first North Korean player in the Se...,https://www.aljazeera.com/news/2019/09/north-k...,https://www.aljazeera.com/mritems/Images/2019/...,2019-09-03T17:25:39Z,"Han Kwang Song, the first North Korean footbal...",0.0,0.0,0.0,7.0,0.0
4,4,bbc-news,BBC News,BBC News,UK government lawyer says proroguing parliamen...,"The UK government's lawyer, David Johnston arg...",https://www.bbc.co.uk/news/av/uk-scotland-4956...,https://ichef.bbci.co.uk/news/1024/branded_new...,2019-09-03T14:39:21Z,NaN,0.0,0.0,0.0,0.0,0.0


In [ ]:
df.shape

(10437, 15)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10437 entries, 0 to 10436
Data columns (total 15 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Unnamed: 0                       10437 non-null  int64  
 1   source_id                        10437 non-null  object 
 2   source_name                      10437 non-null  object 
 3   author                           9417 non-null   object 
 4   title                            10435 non-null  object 
 5   description                      10413 non-null  object 
 6   url                              10436 non-null  object 
 7   url_to_image                     9781 non-null   object 
 8   published_at                     10436 non-null  object 
 9   content                          9145 non-null   object 
 10  top_article                      10435 non-null  float64
 11  engagement_reaction_count        10319 non-null  float64
 12  engagement_comment

In [ ]:
# drop atribut 
df = df.drop(['Unnamed: 0', 'source_id', 'title', 'description', 'source_name', 'author', 'url', 'url_to_image', 'published_at', 'top_article', 'engagement_reaction_count', 'engagement_comment_count', 'engagement_share_count', 'engagement_comment_plugin_count'], axis = 1)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10437 entries, 0 to 10436
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   content  9145 non-null   object
dtypes: object(1)
memory usage: 81.7+ KB


In [ ]:
# Checking duplicate value
print(str(df.describe(include=object)))

                                                  content
count                                                9145
unique                                               8385
top     Chat with us in Facebook Messenger. Find out w...
freq                                                  125


In [ ]:
# remove duplicate value (text) in contenct attribute
df.drop_duplicates(subset=['content'], keep='last')

,content
1,The States jobless rate fell to 5.2 per cent l...
2,Louise Kennedy is showing off her autumn-winte...
3,"Han Kwang Song, the first North Korean footbal..."
5,"""This Tender Land: a Novel"" (Atria Books), by ..."
6,LONDON (Reuters) - The European Union is waiti...
...,...
10429,For his latest installation of the SCAR Projec...
10431,There are numerous bargains waiting to be pick...
10432,Growth in the U.S. economys vast services sect...
10433,ZURICH/HONG KONG (Reuters) - The announcement ...


In [ ]:
# Checking null value in attribute 
df.isna().sum()

content    1292
dtype: int64

In [ ]:
#drop null value
df = df.dropna()


In [ ]:
df

,content
0,WASHINGTON (Reuters) - The National Transporta...
1,The States jobless rate fell to 5.2 per cent l...
2,Louise Kennedy is showing off her autumn-winte...
3,"Han Kwang Song, the first North Korean footbal..."
5,"""This Tender Land: a Novel"" (Atria Books), by ..."
...,...
10428,"Just last week, the Disney Magic arrived to Ne..."
10429,For his latest installation of the SCAR Projec...
10431,There are numerous bargains waiting to be pick...
10432,Growth in the U.S. economys vast services sect...


In [ ]:
#saving the result
df.to_csv("./drive/MyDrive/STBI/Proyek/Dataset/clean_articles_data_word2vec.csv", index=True)

#### C. Text Preprocessing

In [ ]:
# remove punctuation
def punctuation(txt):
  return re.sub(r"[^\w\s]","", str(txt))

df = df['content'].apply(punctuation)

In [ ]:
import nltk
nltk.download('punkt')
# tokenization
def word_tokenize_wrapper(text):
  return word_tokenize(text)
df = df.apply(word_tokenize_wrapper)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
#stopword removal
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
sw_nltk = stopwords.words('english')
print(sw_nltk)
def stopword(text):
  words = [word for word in text if word.lower() not in sw_nltk]
  return words
df = df.apply(stopword)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'bo

In [ ]:
#normalization
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
def lemma(text):
  lemmatizer = WordNetLemmatizer()
  Output= [lemmatizer.lemmatize(words_sent) for words_sent in text]
  return Output
df = df.apply(lemma)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
#saving the rusult
df.to_csv("./drive/MyDrive/STBI/Proyek/Dataset/prepro_articles_data_word2vec.csv", index=True)

#### D. Word Embeding Using Word2Vec

In [ ]:
# vocabulary
docs = df.values
tokenized_docs = df.values
vocab = Counter()
for token in tokenized_docs:
    vocab.update(token)

len(vocab)

35843

In [ ]:
vocab.most_common(10)

[('char', 8773),
 ('said', 1712),
 ('Reuters', 1309),
 ('year', 1099),
 ('US', 1090),
 ('new', 824),
 ('say', 734),
 ('Thursday', 711),
 ('President', 702),
 ('one', 646)]

In [ ]:
#word embedding with word2vec
model1 = Word2Vec(sentences=tokenized_docs, size = 100, workers=1, seed=42)

In [ ]:
# test model
model1.wv.most_similar("moon", topn=5)

[('different', 0.9997158050537109),
 ('making', 0.999668538570404),
 ('usually', 0.9996521472930908),
 ('sometimes', 0.9995822906494141),
 ('problem', 0.9995766878128052)]

In [ ]:
vectorized_docs = vectorize(df, model = model1, strategy="average")
len(vectorized_docs), len(vectorized_docs[0])

(9145, 100)

#### E. Generate and analyze clusters

In [ ]:
def mbkmeans_clusters(X, k, mb=500, print_silhouette_values=False):
    """Generate clusters.

    Args:
        X: Matrix of features.
        k: Number of clusters.
        mb: Size of mini-batches. Defaults to 500.
        print_silhouette_values: Print silhouette values per cluster.

    Returns:
        Trained clustering model and labels based on X.
    """
    km = MiniBatchKMeans(n_clusters=k, batch_size=mb).fit(X)
    print(f"For n_clusters = {k}")
    print(f"Silhouette coefficient: {silhouette_score(X, km.labels_):0.2f}")
    print(f"Inertia:{km.inertia_}")

    if print_silhouette_values:
        sample_silhouette_values = silhouette_samples(X, km.labels_)
        print(f"Silhouette values:")
        silhouette_values = []
        for i in range(k):
            cluster_silhouette_values = sample_silhouette_values[km.labels_ == i]
            silhouette_values.append(
                (
                    i,
                    cluster_silhouette_values.shape[0],
                    cluster_silhouette_values.mean(),
                    cluster_silhouette_values.min(),
                    cluster_silhouette_values.max(),
                )
            )
        silhouette_values = sorted(
            silhouette_values, key=lambda tup: tup[2], reverse=True
        )
        for s in silhouette_values:
            print(
                f"    Cluster {s[0]}: Size:{s[1]} | Avg:{s[2]:.2f} | Min:{s[3]:.2f} | Max: {s[4]:.2f}"
            )
    return km, km.labels_

In [ ]:
clustering, cluster_labels = mbkmeans_clusters(X=vectorized_docs, k=50, print_silhouette_values=True)
df_clusters = pd.DataFrame({
    "text": docs,
    "tokens": [" ".join(text) for text in tokenized_docs],
    "cluster": cluster_labels
})

For n_clusters = 50
Silhouette coefficient: 0.18
Inertia:392.68604791796577
Silhouette values:
    Cluster 2: Size:125 | Avg:1.00 | Min:1.00 | Max: 1.00
    Cluster 4: Size:89 | Avg:1.00 | Min:1.00 | Max: 1.00
    Cluster 41: Size:27 | Avg:0.89 | Min:0.06 | Max: 0.95
    Cluster 7: Size:239 | Avg:0.31 | Min:-0.06 | Max: 0.54
    Cluster 49: Size:19 | Avg:0.30 | Min:-0.04 | Max: 0.55
    Cluster 30: Size:26 | Avg:0.30 | Min:-0.14 | Max: 0.51
    Cluster 26: Size:70 | Avg:0.30 | Min:0.02 | Max: 0.49
    Cluster 8: Size:167 | Avg:0.30 | Min:-0.05 | Max: 0.53
    Cluster 3: Size:79 | Avg:0.25 | Min:-0.00 | Max: 0.46
    Cluster 44: Size:99 | Avg:0.25 | Min:-0.02 | Max: 0.44
    Cluster 24: Size:523 | Avg:0.24 | Min:-0.07 | Max: 0.50
    Cluster 34: Size:116 | Avg:0.24 | Min:-0.03 | Max: 0.46
    Cluster 13: Size:143 | Avg:0.23 | Min:-0.06 | Max: 0.48
    Cluster 42: Size:106 | Avg:0.22 | Min:-0.08 | Max: 0.47
    Cluster 46: Size:102 | Avg:0.21 | Min:-0.06 | Max: 0.48
    Cluster 36: Size:

In [ ]:
print("Top terms per cluster (based on centroids):")
for i in range(50):
    tokens_per_cluster = ""
    most_representative = model1.wv.most_similar(positive=[clustering.cluster_centers_[i]], topn=5)
    for t in most_representative:
        tokens_per_cluster += f"{t[0]} "
    print(f"Cluster {i}: {tokens_per_cluster}")

Top terms per cluster (based on centroids):
Cluster 0: market important there best platform 
Cluster 1: experience list never loan test 
Cluster 2: happening whats Messenger Find Chat 
Cluster 3: Mike Zelensky Secretary adviser Jimmy 
Cluster 4: minute BBC Service World five 
Cluster 5: Twitter intelligence document response policy 
Cluster 6: High Craig Adrian Sir shoe 
Cluster 7: land airport course patient holiday 
Cluster 8: PC Photo Police Media Consumers 
Cluster 9: list experience reason loan decided 
Cluster 10: due Walmart strike reduce cut 
Cluster 11: loss 18 place count deadly 
Cluster 12: 24 le 18 straight deadly 
Cluster 13: president investigate Washington Mike Bolton 
Cluster 14: rule panel approved term hold 
Cluster 15: whistleblower security Bush Ukraines frontrunner 
Cluster 16: concern panel aimed approved close 
Cluster 17: quarterfinal squad qualifying win Japan 
Cluster 18: City Mellon Yankees Mets bourse 
Cluster 19: senior Senate Taliban military Tehran 
Clust

In [ ]:
test_cluster = 48
most_representative_docs = np.argsort(
    np.linalg.norm(vectorized_docs - clustering.cluster_centers_[test_cluster], axis=1)
)
for d in most_representative_docs[:20]:
    print(docs[d])
    print("-------------")

['Getty', 'Images', 'ulliWeWork', 'could', 'slash', 'valuation', 'seek', 'public', 'company', '20', 'billion', 'even', 'postpone', 'IPO', 'according', 'Wall', 'Street', 'JournalliliThe', 'proposed', 'cut', 'come', 'day', 'Journal', 'reported', 'shared', 'officespace', 'co', '2162', 'char']
-------------
['Friday', 'afternoon', 'drove', 'back', 'toward', 'Burgaw', 'town', '4000', '30', 'mile', 'coast', 'check', 'new', 'damage', 'home', 'around', 'house', 'still', 'boarded', 'covered', 'sagging', 'scab', 'blue', 'emergency', 'tarp', 'still', 'win', '1057', 'char']
-------------
['CNNTributes', 'paid', 'Australian', 'sporting', 'great', 'Danny', 'Frawley', 'former', 'Australian', 'Football', 'League', 'AFL', 'player', 'died', 'singlevehicle', 'crash', 'Monday', 'one', 'day', '56th', 'birthday', 'father', 'three', 'went', 'work', 'sport', '2135', 'char']
-------------
['see', 'directing', 'movie', 'near', 'futureTee', 'Florida', 'Im', 'thinking', 'getting', 'closer', 'closer', 'last', 'fil

In [ ]:
docs

array([list(['WASHINGTON', 'Reuters', 'National', 'Transportation', 'Safety', 'Board', 'said', 'Tuesday', 'Tesla', 'Model', 'Autopilot', 'mode', 'struck', 'fire', 'truck', 'Culver', 'City', 'California', 'one', 'series', 'crash', 'board', 'investigating', 'involving', 'Teslas', 'driver', 'assistance', '478', 'char']),
       list(['States', 'jobless', 'rate', 'fell', '52', 'per', 'cent', 'last', 'month', 'according', 'latest', 'official', 'figure', 'higher', 'previously', 'reported', 'account', 'upward', 'revision', 'Central', 'Statistics', 'Office', 'CSO', 'one', 'several', 'last', 'two', 'year', 'Nonethe', '800', 'char']),
       list(['Louise', 'Kennedy', 'showing', 'autumnwinter', 'collection', 'Dublin', 'tonight', 'show', 'start', 'glittering', 'new', 'take', 'Black', 'Watch', 'tartan', 'Sixteen', 'model', 'present', '60', 'ensemble', '200', 'guest', 'including', 'designer', 'close', 'frie', '2156', 'char']),
       ...,
       list(['numerous', 'bargain', 'waiting', 'picked', 'st

#### F. TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

def tfidf(data):
    tfidf = TfidfVectorizer( stop_words='english',use_idf=True)
    tfidf_matrix = tfidf.fit_transform(data)
    return tfidf_matrix

In [ ]:
# Let's create a matrix with tfidf for the column abstract
tfidf_matrix = tfidf(most_representative_docs.values.astype('U'))

AttributeError: ignored

In [ ]:
# in order to explore which documents have more similar respresentaiton, consine simliartiy can be used
from sklearn.metrics.pairwise import linear_kernel
cosine_similarities = linear_kernel(tfidf_matrix[0:1], tfidf_matrix).flatten()

# 10 most related documents indices
related_docs_indices = cosine_similarities.argsort()[:-11:-1]
print("Related Document:",related_docs_indices)

# Cosine Similarties of related documents
print("Cosine Similarites of related documents",cosine_similarities[related_docs_indices])

In [ ]:
# Let's take a look at two most similar document
data.iloc[0]['content']

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
wordcloud = WordCloud().generate(data.iloc[0]['content'])
plt.imshow(wordcloud, interpolation="bilinear")